# జంతు నిపుణుల వ్యవస్థను అమలు చేయడం

[AI for Beginners Curriculum](http://github.com/microsoft/ai-for-beginners) నుండి ఒక ఉదాహరణ.

ఈ నమూనాలో, కొన్ని శారీరక లక్షణాల ఆధారంగా జంతువును గుర్తించడానికి ఒక సులభమైన జ్ఞానాధారిత వ్యవస్థను అమలు చేస్తాము. ఈ వ్యవస్థను క్రింది AND-OR చెట్టు ద్వారా ప్రదర్శించవచ్చు (ఇది మొత్తం చెట్టు యొక్క ఒక భాగం, మేము సులభంగా మరిన్ని నియమాలను జోడించవచ్చు):

![](../../../../translated_images/AND-OR-Tree.5592d2c70187f283703c8e9c0d69d6a786eb370f4ace67f9a7aae5ada3d260b0.te.png)


## మన స్వంత నిపుణుల వ్యవస్థ షెల్ వెనుకబడిన నిర్ధారణతో

ఉత్పత్తి నియమాల ఆధారంగా జ్ఞాన ప్రాతినిధ్యం కోసం ఒక సరళమైన భాషను నిర్వచించడానికి ప్రయత్నిద్దాం. నియమాలను నిర్వచించడానికి Python తరగతులను కీవర్డ్లుగా ఉపయోగిస్తాము. మూడే రకాల తరగతులు ఉంటాయి:
* `Ask` అనేది వినియోగదారుని అడగవలసిన ప్రశ్నను సూచిస్తుంది. ఇది సాధ్యమైన సమాధానాల సమాహారాన్ని కలిగి ఉంటుంది.
* `If` అనేది ఒక నియమాన్ని సూచిస్తుంది, ఇది నియమపు విషయాన్ని నిల్వ చేయడానికి ఒక సింటాక్టిక్ షుగర్ మాత్రమే.
* `AND`/`OR` అనేవి చెట్టు యొక్క AND/OR శాఖలను సూచించే తరగతులు. అవి కేవలం లోపల ఆర్గ్యుమెంట్ల జాబితాను నిల్వ చేస్తాయి. కోడ్ సులభతరం చేయడానికి, అన్ని ఫంక్షనాలిటీని తల్లిదండ్రి తరగతి `Content` లో నిర్వచించబడింది.


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

మన వ్యవస్థలో, వర్కింగ్ మెమరీలో **గుణం-విలువ జంటలుగా** **వాస్తవాలు** జాబితా ఉంటుంది. జ్ఞానాధారం అనేది ఒక పెద్ద నిఘంటువు లాగా నిర్వచించవచ్చు, ఇది చర్యలను (వర్కింగ్ మెమరీలో చేర్చవలసిన కొత్త వాస్తవాలు) AND-OR వ్యక్తీకరణలుగా వ్యక్తం చేసిన షరతులకు మ్యాప్ చేస్తుంది. అలాగే, కొన్ని వాస్తవాలను `Ask` చేయవచ్చు.


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

వెనుకకు సూచన చేయడానికి, మనం `Knowledgebase` క్లాస్‌ను నిర్వచిస్తాము. ఇది కలిగి ఉంటుంది:
* పని చేసే `memory` - లక్షణాలను విలువలకు మ్యాప్ చేసే డిక్షనరీ
* పైగా నిర్వచించిన ఫార్మాట్‌లో ఉన్న Knowledgebase `rules`

రెండు ప్రధాన విధానాలు:
* `get` ఒక లక్షణం యొక్క విలువను పొందడానికి, అవసరమైతే సూచన చేస్తుంది. ఉదాహరణకు, `get('color')` రంగు స్లాట్ యొక్క విలువను పొందుతుంది (అవసరమైతే అడుగుతుంది, మరియు తరువాత ఉపయోగానికి పని చేసే మెమరీలో విలువను నిల్వ చేస్తుంది). మనం `get('color:blue')` అడిగితే, అది రంగును అడిగి, ఆ రంగు ఆధారంగా `y`/`n` విలువను ఇస్తుంది.
* `eval` వాస్తవ సూచనను నిర్వహిస్తుంది, అంటే AND/OR ట్రీని దాటుతూ, ఉప-లక్ష్యాలను మూల్యాంకనం చేస్తుంది, మొదలైనవి.


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

ఇప్పుడు మన జంతు జ్ఞానాధారాన్ని నిర్వచించి సంప్రదింపును నిర్వహిద్దాం. ఈ కాల్ మీకు ప్రశ్నలు అడుగుతుంది. అవును-కాదు ప్రశ్నలకు మీరు `y`/`n` టైప్ చేసి సమాధానం ఇవ్వవచ్చు, లేదా ఎక్కువ ఎంపికలున్న ప్రశ్నలకు 0..N సంఖ్యను పేర్కొనవచ్చు.


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## ఫార్వర్డ్ ఇన్ఫరెన్స్ కోసం PyKnow ఉపయోగించడం

తరువాతి ఉదాహరణలో, జ్ఞాన ప్రాతినిధ్యం కోసం ఉన్న లైబ్రరీలలో ఒకటి అయిన [PyKnow](https://github.com/buguroo/pyknow/) ఉపయోగించి ఫార్వర్డ్ ఇన్ఫరెన్స్‌ను అమలు చేయడానికి ప్రయత్నిస్తాము. **PyKnow** అనేది Pythonలో ఫార్వర్డ్ ఇన్ఫరెన్స్ సిస్టమ్స్ సృష్టించడానికి ఉపయోగించే లైబ్రరీ, ఇది పాత క్లాసికల్ సిస్టమ్ [CLIPS](http://www.clipsrules.net/index.html)కి సమానంగా రూపొందించబడింది.

మనం ఫార్వర్డ్ చైనింగ్‌ను స్వయంగా కూడా అమలు చేయవచ్చు, కానీ సాధారణ అమలు పద్ధతులు ఎక్కువగా సమర్థవంతంగా ఉండవు. మరింత సమర్థవంతమైన రూల్ మ్యాచ్ చేయడానికి ప్రత్యేక అల్గోరిథం [Rete](https://en.wikipedia.org/wiki/Rete_algorithm) ఉపయోగిస్తారు.


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/buguroo/pyknow/

  Cloning https://github.com/buguroo/pyknow/ to /tmp/pip-req-build-3cqeulyl
  Running command git clone --filter=blob:none --quiet https://github.com/buguroo/pyknow/ /tmp/pip-req-build-3cqeulyl
  Resolved https://github.com/buguroo/pyknow/ to commit 48818336f2e9a126f1964f2d8dc22d37ff800fe8
  Preparing metadata (setup.py) ... done
  Using cached frozendict-1.2.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... done
  Using cached schema-0.6.7-py2.py3-none-any.whl (14 kB)
  Created wheel for pyknow: filename=pyknow-1.7.0-py3-none-any.whl size=34228 sha256=b7de5b09292c4007667c72f69b98d5a1b5f7324ff15f9dd8e077c3d5f7aade42
  Stored in directory: /tmp/pip-ephem-wheel-cache-k7jpave7/wheels/81/1a/d3/f6c15dbe1955598a37755215f2a10449e7418500d7bd4b9508
  Created wheel for frozendict: filename=frozendict-1.2-py3-none-any.whl size=3148 sha256=2863d55c240d2409cddf05ccfe600591f8478681549fc97555c47c90dc6bb160
  Stored in directory: /home/rg/.cache/pip/wheels/49/ac/f8/cb8120244e710bdb479c86198b03c7b08c

In [13]:
from pyknow import *
#import pyknow

మేము మా సిస్టమ్‌ను `KnowledgeEngine` ను సబ్‌క్లాస్ చేసే క్లాస్‌గా నిర్వచిస్తాము. ప్రతి నియమం `@Rule` అనోటేషన్‌తో వేరే ఫంక్షన్ ద్వారా నిర్వచించబడుతుంది, ఇది నియమం ఎప్పుడు అమలవ్వాలో సూచిస్తుంది. నియమం లోపల, మేము `declare` ఫంక్షన్ ఉపయోగించి కొత్త వాస్తవాలను జోడించవచ్చు, మరియు ఆ వాస్తవాలను జోడించడం ఫార్వర్డ్ ఇన్ఫరెన్స్ ఇంజిన్ ద్వారా మరిన్ని నియమాలు పిలవబడటానికి కారణమవుతుంది.


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

ఒకసారి మనం ఒక జ్ఞానాధారం నిర్వచించిన తర్వాత, మన పని జ్ఞాపకశక్తిని కొన్ని ప్రారంభ వాస్తవాలతో నింపుతాము, మరియు ఆపై `run()` పద్ధతిని పిలిచి తర్కాన్ని నిర్వహిస్తాము. మీరు ఫలితంగా చూడవచ్చు కొత్త తర్కించిన వాస్తవాలు పని జ్ఞాపకశక్తికి జోడించబడ్డాయి, అందులో ఆ జంతువు గురించి తుది వాస్తవం కూడా ఉంటుంది (మనం అన్ని ప్రారంభ వాస్తవాలను సరిగ్గా సెట్ చేసినట్లయితే).


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**అస్పష్టత**:  
ఈ పత్రాన్ని AI అనువాద సేవ [Co-op Translator](https://github.com/Azure/co-op-translator) ఉపయోగించి అనువదించబడింది. మేము ఖచ్చితత్వానికి ప్రయత్నించినప్పటికీ, ఆటోమేటెడ్ అనువాదాల్లో పొరపాట్లు లేదా తప్పిదాలు ఉండవచ్చు. మూల పత్రం దాని స్వదేశీ భాషలోనే అధికారిక మూలంగా పరిగణించాలి. ముఖ్యమైన సమాచారానికి, ప్రొఫెషనల్ మానవ అనువాదం సిఫార్సు చేయబడుతుంది. ఈ అనువాదం వాడకంలో ఏర్పడిన ఏవైనా అపార్థాలు లేదా తప్పుదారుల కోసం మేము బాధ్యత వహించము.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
